# Centrality

In network analysis, indicators of **centrality** identify the most important vertices within a graph.

Applications:

- Identifying the most influential person(s) in a social network
  
- Key infrastructure nodes in the Internet
  
- Super-spreaders of disease.

# Betweenness Centrality

<b>Betweenness centrality</b>, in Grapgh Theory, is a measure of centrality in a graph based on shortest paths. Vertices with high betweenness may have considerable influence within a network by virtue of their control over information passing between others. They are also the ones whose removal from the network will most disrupt communications between other vertices because they lie on the largest number of paths taken by messages.

It was introduced as a measure for quantifying the control of a human on
the communication between other humans in a social network by **Linton Freeman**. 
In his conception, vertices that have a high probability to occur on a randomly chosen **shortest path** 
between two randomly chosen vertices have a high betweenness.

The betweenness of a vertex $v$ in a graph $G := ( V , E ) $ with $V$ vertices is computed as follows:

1. For each pair of vertices $(s,t)$, compute the shortest paths between them.
  
2. For each pair of vertices $(s,t)$, determine the number of shortest paths that pass through the vertex in question (here, vertex $v$).
  
3. Sum this fraction over all pairs of vertices $(s,t)$.

More compactly the betweenness can be represented as:

$\sum_{s\neq v\neq t \in V} \frac{\sigma_{st}(v)}{\sigma_{st}}$

where $\sigma_{st}$ is total number of shortest paths from node $s$ to node $t$ and $\sigma_{st}(v)$ is the number of those paths that pass through $v$.